<a href="https://colab.research.google.com/github/CurtCalledBurt/DS-Unit-2-Applied-Modeling/blob/master/module2/assignment_applied_modeling_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

df = pd.read_csv('/Users/curtismckendrick/Downloads/games.csv')

In [0]:
df.isna().sum()

id                       0
type                     0
name                    41
yearpublished            3
minplayers               3
maxplayers               3
playingtime              3
minplaytime              3
maxplaytime              3
minage                   3
users_rated              0
average_rating           0
bayes_average_rating     0
total_owners             0
total_traders            0
total_wanters            0
total_wishers            0
total_comments           0
total_weights            0
average_weight           0
dtype: int64

In [0]:
# feature selection

# We try predicting 'average_rating' and see what happens with that.

target = 'average_rating'

# Obviously we wouldn't know the bayes_average_rating if we knew the average rating, so we remove that from
# the features. And I'm considering removing the 2 weight columns again, because I still have no idea what 
# they are. 

things_we_cannot_know = ['bayes_average_rating']

things_we_do_not_need = ['name', 'type', 'id']

features = df.columns.drop([target] + things_we_cannot_know + things_we_do_not_need)
X = df[features]
y = df[target]

In [0]:
# train/test/val split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2019)

In [0]:
# train/test/val split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=2019)
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

((45738, 15), (45738,), (20328, 15), (20328,), (15246, 15), (15246,))

In [0]:
# quick and dirty baseline regressor

def baseline_Regressor(target):
  from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

  # making a list of the same size as 'target' 
  # with the mean of the target as the only possible entree
  mean_target = target.mean()
  pred_baseline = [mean_target] * len(target)
  
  #calculating baseline error and score
  mae = mean_absolute_error(target, pred_baseline)
  mse = mean_squared_error(target, pred_baseline)
  r2 = r2_score(target, pred_baseline)
  
  # printing the errors and scores
  print('Baseline MAE: ', mae)
  print('Baseline MSE: ', mse)
  print('Baseline R^2: ', r2)

In [0]:
# get a list of baseline predictions
y_pred_base = baseline_Regressor(y_train)

Baseline MAE:  2.711469150949111
Baseline MSE:  9.321468469749549
Baseline R^2:  0.0


In [0]:
# more robust modelling
# we'll start with a random forest before making our way to xgboost
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    RandomForestRegressor(n_estimators=150, random_state=2019, n_jobs=-1)
)
# Fit the model
pipeline.fit(X_train, y_train);

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


#make a prediction on the train set
y_pred = pipeline.predict(X_train)

# MAE of train set
mae = mean_absolute_error(y_train, y_pred)
print('MAE Train Error: ', mae)

# RMSE of train set
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print('RMSE Train Error: ', rmse)

# R2 score of train set
r2 = r2_score(y_train, y_pred)
print('Train R^2 Score: ', r2)


print('\n')


# make prediction on the val set
y_pred = pipeline.predict(X_val)

# MAE of val set
mae = mean_absolute_error(y_val, y_pred)
print('MAE Val Error: ', mae)

# RMSE of val set
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print('RMSE Val Error: ', rmse)

# R2 score of val set
r2 = r2_score(y_val, y_pred)
print('Val R^2 Score: ', r2)

MAE Train Error:  0.225298161233403
RMSE Train Error:  0.39027141167513846
Train R^2 Score:  0.9836601094274797


MAE Val Error:  0.5841072198455923
RMSE Val Error:  0.9932021499061355
Val R^2 Score:  0.8945718078292232


In [0]:
from xgboost import XGBRegressor

imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.fit_transform(X_train)

model = XGBRegressor(objective='reg:squarederror',
            n_estimators=1000, 
            random_state=2019,
            learning_rate = 0.5,
            n_jobs=-1
            )

eval_set = [ 
    (X_train_imputed, y_train), 
    (X_val_imputed, y_val) 
           ]

model.fit(X_train_imputed, y_train,
         eval_set=eval_set,
         eval_metric= 'mae',
         early_stopping_rounds=50);

[0]	validation_0-mae:2.06505	validation_1-mae:2.06072
Multiple eval metrics have been passed: 'validation_1-mae' will be used for early stopping.

Will train until validation_1-mae hasn't improved in 50 rounds.


//anaconda3/envs/unit2real/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[1]	validation_0-mae:1.19788	validation_1-mae:1.19436
[2]	validation_0-mae:0.859284	validation_1-mae:0.851297
[3]	validation_0-mae:0.738382	validation_1-mae:0.728952
[4]	validation_0-mae:0.695918	validation_1-mae:0.686167
[5]	validation_0-mae:0.678682	validation_1-mae:0.667681
[6]	validation_0-mae:0.676036	validation_1-mae:0.665205
[7]	validation_0-mae:0.673325	validation_1-mae:0.6621
[8]	validation_0-mae:0.66849	validation_1-mae:0.65679
[9]	validation_0-mae:0.664786	validation_1-mae:0.65261
[10]	validation_0-mae:0.665485	validation_1-mae:0.653927
[11]	validation_0-mae:0.664932	validation_1-mae:0.653505
[12]	validation_0-mae:0.662736	validation_1-mae:0.65128
[13]	validation_0-mae:0.662426	validation_1-mae:0.651447
[14]	validation_0-mae:0.662511	validation_1-mae:0.651245
[15]	validation_0-mae:0.660838	validation_1-mae:0.649447
[16]	validation_0-mae:0.659079	validation_1-mae:0.647893
[17]	validation_0-mae:0.657262	validation_1-mae:0.646422
[18]	validation_0-mae:0.655943	validation_1-mae:

In [0]:
y_pred = model.predict(X_train_imputed)
print('Train MAE: ', mean_absolute_error(y_train, y_pred))
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_pred)))
print('Train R2: ', r2_score(y_train, y_pred))

print('\n')

y_pred = model.predict(X_val_imputed)
print('Val MAE: ', mean_absolute_error(y_val, y_pred))
print('Val RMSE: ', np.sqrt(mean_squared_error(y_val, y_pred)))
print('Val R2: ', r2_score(y_val, y_pred))

Train MAE:  0.6092844171745581
Train RMSE:  0.9828667149384276
Train R2:  0.8963653653425047


Val MAE:  0.619484039820385
Val RMSE:  0.9921008963533039
Val R2:  0.8948054738641488
